# **Step1: Install amd import libraries**

In [1]:
!pip install -qq "transformers==4.35" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "tiktoken"
!pip install -qq gradio
!pip install -qq gtts

!pip install sentence_transformers
!pip install faiss-cpu
!pip install datasets

import pandas as pd
import gradio as gr
from gtts import gTTS
import os
import torch
from datasets import Dataset, load_dataset
from random import randrange
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from peft import AutoPeftModelForCausalLM
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import random
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np

print(transformers.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device ',device,' is being used')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


4.35.0
Device  cuda  is being used


# **Step 2: Load Saved Model & Questions Embedding**

In [6]:
# load model
!gdown 1KCFWGmIyphbLiHxmrcuTm1wHE43W91Fx

# load questions embedding
!gdown 1hoQoV8daHWtsYZ5znxXLg1OpP0-CF60E

Downloading...
From (original): https://drive.google.com/uc?id=1KCFWGmIyphbLiHxmrcuTm1wHE43W91Fx
From (redirected): https://drive.google.com/uc?id=1KCFWGmIyphbLiHxmrcuTm1wHE43W91Fx&confirm=t&uuid=ed2c39f3-4ffb-4f95-ac38-376908af44c8
To: /content/RAG_SBERT_faiss_T5.zip
100% 225M/225M [00:10<00:00, 21.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1hoQoV8daHWtsYZ5znxXLg1OpP0-CF60E
From (redirected): https://drive.google.com/uc?id=1hoQoV8daHWtsYZ5znxXLg1OpP0-CF60E&confirm=t&uuid=d6665e31-0091-41cb-80c8-b4845d50c619
To: /content/question_embeddings.npy
100% 153M/153M [00:09<00:00, 16.8MB/s]


In [7]:
!unzip /content/RAG_SBERT_faiss_T5.zip

Archive:  /content/RAG_SBERT_faiss_T5.zip
  inflating: RAG_SBERT_faiss_T5/added_tokens.json  
  inflating: RAG_SBERT_faiss_T5/generation_config.json  
  inflating: RAG_SBERT_faiss_T5/config.json  
  inflating: RAG_SBERT_faiss_T5/tokenizer_config.json  
  inflating: RAG_SBERT_faiss_T5/special_tokens_map.json  
  inflating: RAG_SBERT_faiss_T5/spiece.model  
  inflating: RAG_SBERT_faiss_T5/model.safetensors  


##2-1 Load tokenizer

In [15]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Define the path to the saved model
model_path = "/content/RAG_SBERT_faiss_T5"

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_path)

# Load the model
model = T5ForConditionalGeneration.from_pretrained(model_path)
model.to(device)

# Verify the model and tokenizer have been loaded correctly
print("Saved tokenizer and model loaded successfully")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Saved tokenizer and model loaded successfully


In [ ]:
# # Training parameters
# epochs = 2
# batch_size = 1
# learning_rate = 5e-5
# adam_epsilon = 1e-8
# warmup_steps = 0
# train_questions = 91772


# # Load the T5 tokenizer and model
# model_name = 't5-base'
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)

# model.to(device)

# # Optimizer and scheduler
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon)
# total_steps = (train_questions) // batch_size * epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

# def load_checkpoint(checkpoint_path, model, optimizer, scheduler):
#     checkpoint = torch.load(checkpoint_path,  map_location=torch.device(device))
#     model.load_state_dict(checkpoint['model_state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#     scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
#     start_epoch = checkpoint['epoch'] + 1  # start from the next epoch
#     loss = checkpoint['loss']
#     return model, optimizer, scheduler, start_epoch, loss

# # Initialize variables
# start_epoch = 0  # default start epoch
# checkpoint_path = '/content/checkpoint_epoch_1_batch_40000.pt'

# model, optimizer, scheduler, start_epoch, loss = load_checkpoint(checkpoint_path, model, optimizer, scheduler)
# print(f"Loaded checkpoint from epoch {start_epoch}, loss: {loss}")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

Loaded checkpoint from epoch 2, loss: 75456.31543746311


# **Step 3: Load Dataset for creating contexts using questions embedding**

In [9]:
# Prepare data
def organize_dataset(dataset):
  questions = dataset['title']
  questions_selftext = dataset['selftext']
  categories = dataset['category']

  answers = []
  answers_scores = []
  for item in dataset:
    answers.append(item['answers']['text'])
    answers_scores.append(item['answers']['score'])

  dataset_dict = {"questions":questions, "selftext": questions_selftext, "category":categories, "answers":answers, "score":answers_scores}
  return dataset_dict

dataset = load_dataset("eli5_category")

train_dataset_dict = organize_dataset(dataset['train'])
valid1_dataset_dict = organize_dataset(dataset['validation1'])
valid2_dataset_dict = organize_dataset(dataset['validation2'])
test_dataset_dict = organize_dataset(dataset['test'])

train_questions = train_dataset_dict['questions']
train_answers = train_dataset_dict['answers']
train_scores = train_dataset_dict['score']

valid_questions = valid1_dataset_dict['questions']
valid_answers = valid1_dataset_dict['answers']
valid_scores = valid1_dataset_dict['score']

all_questions = train_dataset_dict['questions'] + valid1_dataset_dict['questions'] + valid2_dataset_dict['questions']
all_answers = train_dataset_dict['answers'] + valid1_dataset_dict['answers'] + valid2_dataset_dict['answers']
all_scores = train_dataset_dict['score'] + valid1_dataset_dict['score'] + valid2_dataset_dict['score']


# Load models
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [10]:
# Define weighted sampling function
def weighted_sample(answers, scores, k=3):
    k = min(k, len(answers))

    if len(answers) == 0:
        return []

    total_score = sum(scores)
    probabilities = [score / total_score for score in scores]
    sampled_indices = np.random.choice(len(answers), size=k, replace=False, p=probabilities)
    return [answers[i] for i in sampled_indices]

def index_Q_embeddings(question_embeddings):
    # Index embeddings using FAISS
    index = faiss.IndexFlatL2(question_embeddings.shape[1])
    index.add(question_embeddings.cpu().numpy())
    return index

def load_Q_embeddings(train_question_embeddings_file='/kaggle/working/question_embeddings.npy'):
    # Load the embeddings from the file
    train_questions_embeddings = torch.tensor(np.load(train_question_embeddings_file))
    print("Loaded Question Embeddings.")
    return train_questions_embeddings

def find_relevant_context_SBERT(question, question_embeddings, answers, scores, index, use='train'):
    '''
    this function returns top-2 similar answers for training phase
    - top-2 answers of similar questions
    - top-2 answers of current question
    '''
    new_question_embedding = sbert_model.encode(question, convert_to_tensor=True, show_progress_bar=False).unsqueeze(0)
    new_question_embedding = torch.nn.functional.normalize(new_question_embedding, p=2, dim=1)

    k = 5  # Retrieve a larger number of candidates
    D, I = index.search(new_question_embedding.cpu().numpy(), k)

    threshold = 0.5
    filtered_indices = []
    for i, distance in zip(I[0], D[0]):
        candidate_embedding = question_embeddings[i].cpu().numpy().reshape(1, -1)
        similarity = cosine_similarity(new_question_embedding.cpu().numpy(), candidate_embedding)[0][0]
        if similarity >= threshold:
            filtered_indices.append(i)

    if (len(filtered_indices) == 0):
        if(use == 'test'):
            return [""], [""]
        else:
            n_index = all_questions.index(question)
            top_ans = weighted_sample(answers[n_index], scores[n_index], k=3)
            return top_ans, [""]
    else:
        # Retrieve relevant contexts (questions and answers)
        relevant_contexts = [weighted_sample(answers[i], scores[i], k=3) for i in filtered_indices[1:]]
        if len(relevant_contexts) >= 1:  # when no relevant context is found
            relevant_contexts = relevant_contexts[0]
        top_2_ans = weighted_sample(answers[filtered_indices[0]], scores[filtered_indices[0]], k=3)
        return top_2_ans, relevant_contexts

# Load questions embedding
question_embeddings = load_Q_embeddings("/content/question_embeddings.npy")
faiss_index = index_Q_embeddings(question_embeddings)

Loaded Question Embeddings.


# **Step 4: Generate answer**

In [44]:
def answer_query(model, tokenizer, query,temperature=0.6,long_answer = False):

    context = find_relevant_context_SBERT(query, question_embeddings, all_answers, all_scores, faiss_index, use='test')
    prompt = "question: {} context: {}".format(query, ' '.join(context[0] + context[1]))

    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

    if long_answer:

        outputs = model.generate(input_ids=input_ids,


                             min_length=100,
                             max_new_tokens=350,
                                 #repetition_penalty=0.2,
                            #  do_sample=True,
                            #  top_p=0.9,
                             temperature=temperature)

    else:

        outputs = model.generate(input_ids=input_ids,


                             min_length=40,
                             #max_length = 100,
                             max_new_tokens=100,
                                 #repetition_penalty=0.2,
                            #  do_sample=True,
                            #  top_p=0.9,
                             temperature=temperature)
    answer = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

    return prompt,answer

In [40]:
# Remove question from answer and remove last uncomplete sentence
def pretty_asnwer(prompt,answer):
    new_answer = answer
    if not(new_answer.endswith('.') or new_answer.endswith('!') or new_answer.endswith('?')):
        #print('Not complete answer')
        # last_index_of_question = len(prompt)
        # new_answer = new_answer[last_index_of_question:]
        last_sentence = new_answer.split('. ')[-1]
        new_answer = new_answer.replace(last_sentence, " ")
        if new_answer.find('Answer')>-1:
          #print('here')
          index = new_answer.find('Answer')
          new_answer = new_answer[index+8:]



    return new_answer


# **Step 5: Ueser Interface**

In [45]:
# Check availabilty:

def text_to_speech(text):
        # Convert text to speech
        tts = gTTS(text)
        # Save the converted audio to a file
        tts.save("output.mp3")
        # Return the file path
        return "output.mp3"

def aks_question(Question, Long_Format, Audio_Play ):
    query = Question

    #check Long format

    if Long_Format == True:

        prompt,long_answer = answer_query(model, tokenizer, query,temperature=0.6,long_answer = True)
        long_answer = pretty_asnwer(prompt,long_answer)

        if Audio_Play:
          text = long_answer
          audio_ouput = text_to_speech(text)
          return long_answer, audio_ouput

        else:
          text = ' You did not chose audio play'
          audio_ouput = text_to_speech(text)
          return long_answer, audio_ouput

    else:
        text = ' Please check one checkbox at least'
        audio_ouput = text_to_speech(text)
        return 'Please check one checkbox at least', audio_ouput


demo = gr.Interface(
    fn=aks_question,

    inputs=["text", "checkbox","checkbox"],

    outputs=[
        gr.Textbox(label="Long Answer"),
        gr.Audio(label='Long_Format_Audio_Play')
    ]
)
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b8c6a4426a15e93e3b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
